In [1]:
from tools import solution_xls_extract_hfc_replacement as sxe  # The extraction pipeline custom made for HFCReplacement
from tools import create_expected_zip as cez
from tools import expected_ghost
from solution import factory
from pathlib import Path
import pandas as pd
import openpyxl
import importlib

In [2]:
excelfile = Path(r"C:\Users\Daniel\repos\RRS_HFCReplacement Model_June2020Integration.xlsm")
outdir = Path(r"C:\Users\Daniel\repos\solutions\solution\hfc_replacement")
outdir.mkdir(parents=True, exist_ok=True)

In [3]:
importlib.reload(sxe)

<module 'tools.solution_xls_extract_hfc_replacement' from 'C:\\Users\\Daniel\\repos\\solutions\\tools\\solution_xls_extract_hfc_replacement.py'>

In [4]:
sxe.output_solution_python_file(outputdir=outdir, xl_filename=str(excelfile))

C:\Users\Daniel\anaconda3\envs\pd-dev\lib\site-packages\openpyxl\reader\drawings.py:59: UserWarning: wmf image format is not supported so the image is being dropped
  warn(msg)
C:\Users\Daniel\anaconda3\envs\pd-dev\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
C:\Users\Daniel\anaconda3\envs\pd-dev\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\Daniel\anaconda3\envs\pd-dev\lib\site-packages\openpyxl\reader\drawings.py:29: UserWarning: DrawingML support is incomplete and limited to charts and images only. Shapes and drawings will be lost.
  warn("DrawingML support is incomplete and limited to charts and images only. Shapes and drawings will be lost.")
C:\Users\Daniel\repos\solutions\tools\solution_xls_extract_hfc_replacement.py:906: SettingWithCopyWarning: 
A value is trying to be set o

NOTE DMK: 07.08.21 extraction code runs.

I'll collect ISSUES here.
- ad is not extracted at all. They look very different from the others. They are in a sheet called "Adoption Calcs" in excel.

What WORKED?
- The two TAM scenarios and their mean were correctly converted to .csv from the TAM Data sheet in excel.
- Python ac in ScenarioRecord tab in excel according to docs.
- The three scenario names in ScenarioRecord are recognized correctly.

What is UNCLEAR?
- ca_pds_data looks good but I cannot find the numbers in the excel.

NOTES!
- VMA stands for Variable Meta Analysis. The result of VMA is a collection of scaler parameters of the model. There can be low, high or average versions of the parameter.
- RRS models deal with reduction and replacement. HFCReplacement is a RRS model and importantly models adoption as a proportion of the total addressable market (TAM).
- TAM stands for total addressable market. The unit of TAM is kton refrigerant.
- ODS stands for ozone-depleting substances.
- Two adoption scenarios and their average are in "Adoption Calcs" there is not "Adoption Data" sheet. The automated function looks for "Adoption Data"
- "Advanced Control" sheet is where researchers input settings and in Python those are translated to ac subdirectory.
- "Variable Meta-analysis-Open" sheet exists but there are barely any entries, only adoption data for Velders et al. (2017). The automated extraction function does not extract any of this data for unknown reasons.

solution_xls_extract.py FUNCTIONS!
* get_rrs_scenarios(wb, solution_category)
    - Extracts data from the ScenarioRecord sheet as it is stored in the ac subdirectory.
    - Needs fixing, most values in ac are 0.

In [8]:
(constructor,scenarios) = factory.one_solution_scenarios("hfc_replacement")
myscenario = constructor(scenarios[1])
bi_co2e = myscenario.c2.co2eq_mmt_reduced()
bi_co2e

ZeroDivisionError: float division by zero

In [5]:
scenarios[0]

'PDS1-67p2050-LowerLowGWP Adoption'

In [10]:
wb = openpyxl.load_workbook(excelfile, data_only=False)

C:\Users\Daniel\anaconda3\envs\pd-dev\lib\site-packages\openpyxl\reader\drawings.py:59: UserWarning: wmf image format is not supported so the image is being dropped
  warn(msg)
C:\Users\Daniel\anaconda3\envs\pd-dev\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
C:\Users\Daniel\anaconda3\envs\pd-dev\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\Daniel\anaconda3\envs\pd-dev\lib\site-packages\openpyxl\reader\drawings.py:29: UserWarning: DrawingML support is incomplete and limited to charts and images only. Shapes and drawings will be lost.
  warn("DrawingML support is incomplete and limited to charts and images only. Shapes and drawings will be lost.")


In [35]:
a = wb['Adoption Calcs']

In [36]:
col_letters = 'GHI'
col_list = []
for letter in col_letters:
    col_list.append(a[letter])

In [31]:
col_list[0][73].value

1522.5059686228744

In [37]:
# TODO put into script
df = pd.read_excel(wb, engine='openpyxl', sheet_name='Adoption Calcs',
header=10, index_col=0, usecols=[0,4,5,8], nrows=83)

In [35]:
df.index

Int64Index([1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036], dtype='int64', name='Year')

In [40]:
df.columns = [*df.columns[:2], "Mean lowGWP"]

In [44]:
df['lowGWP-upper'] /= 1000 

In [38]:
df = df.loc[2012:]
df

,lowGWP-lower,lowGWP-upper,Unnamed: 8
Year,,,
2012,0.0,0.0,0
2013,1167.8,1161.3,1.16455
2014,2800.4,2780.4,2.7904
2015,5564.9,5523.9,5.5444
2016,8740.7,8711.2,8.72595
2017,12343.9,12361.5,12.3527
2018,16393.0,16496.3,16.44465
2019,38855.2,40786.2,39.8207
2020,58857.9,62347.6,60.60275


In [41]:
    missing_rows = range(73,83)
    for idx, row in enumerate(missing_rows):
        df['lowGWP-lower'][2051+idx] = a['G'+str(row)].value
        df['lowGWP-upper'][2051+idx] = a['G'+str(row)].value

In [42]:
df

,lowGWP-lower,lowGWP-upper,Unnamed: 8
Year,,,
2012,0.0,0.0,0
2013,1167.8,1161.3,1.16455
2014,2800.4,2780.4,2.7904
2015,5564.9,5523.9,5.5444
2016,8740.7,8711.2,8.72595
2017,12343.9,12361.5,12.3527
2018,16393.0,16496.3,16.44465
2019,38855.2,40786.2,39.8207
2020,58857.9,62347.6,60.60275


In [13]:
wb['Variable Meta-analysis-Open']

<Worksheet "Variable Meta-analysis-Open">